In [1]:
import requests
import os

NOTION_API_KEY = <your_key>
COOKING_ID = "013eb25bb1444da6a8a842fba94b3f0e"

In [2]:
db_data = requests.post(
    f"https://api.notion.com/v1/databases/{COOKING_ID}/query",
    headers={
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Notion-Version": "2022-06-28",
        "Content-Type": "application/json",
    },
).json()

In [13]:
recipe_data = db_data['results'][0]
recipe_properties = recipe_data['properties']
recipe_name = recipe_properties['Name']['title'][0]['plain_text']
print(recipe_name, recipe_data['id'])

Карбонара 070bc3c1-b37a-4e04-ad1a-ba9311eea771


In [16]:
f"https://api.notion.com/v1/blocks/{recipe_data['id']}/children"

'https://api.notion.com/v1/blocks/070bc3c1-b37a-4e04-ad1a-ba9311eea771/children'

In [42]:
def get_recipe_text(recipe_id):
    recipe_page_data = requests.get(
        f"https://api.notion.com/v1/blocks/{recipe_id}/children",
        headers={
            "Authorization": f"Bearer {NOTION_API_KEY}",
            "Notion-Version": "2022-06-28",
        },
    ).json()

    content_lines = []

    for block in recipe_page_data['results']:
        block_type = block['type']
        assert block_type in ['paragraph', 'bulleted_list_item'], f"Can't handle {block_type} yet"
        
        block_rich_text = block[block_type]['rich_text']
        if len(block_rich_text) == 0:
            content_lines.append('')
            continue

        cur_line = ''
        if block_type == 'bulleted_list_item':
            cur_line += '- '
        cur_line += block_rich_text[0]['plain_text']
        content_lines.append(cur_line)
    
    return '\n'.join(content_lines)

In [44]:
print(get_recipe_text(recipe_data['id']))

Бекона нарезать и обжарить
(тонкие куски жарятся быстрее, куски потолще надо жарить на медленном и подольше)
Отварить макарошки (за ~минуту до полной готовности), взять кружку крахмальной воды
Макарошки в дуршлаг, потом к нагретому бекону, немного пожарить их так, потом залить крахмальной водой
Это залить соусом карбонарным (2/3 блока пармезана к 3 желткам и 1 яйцу)

Маша любит:
- Бекон не слишком угольки
- Сыра побольше
- ЖИР ИЗ БЕКОНА УБРАТЬ
- Макарошки желательно мелкие трубочки. А если спагетти, то в 4x укоротить 
